## 3. Data Analysis: Investigating BNPL Stock Returns and Monetary Policy

Do BNPL stock returns respond to changes in monetary policy? The sector's rapid growth and reliance on funding markets motivate this question. According to the Consumer Financial Protection Bureau's 2025 report, BNPL adoption has grown substantially, with 21% of consumers with credit records utilizing BNPL services in 2022. The sector's reliance on short-term funding suggests that interest rate changes should affect BNPL firms' costs and profitability, which should be reflected in stock returns.

The analysis proceeds in several steps. BNPL companies are identified through regulatory reports, financial data platforms, and industry analysis. Stock price data are collected from Yahoo Finance, and macroeconomic data are gathered from FRED. Variables are constructed and relationships examined through visual analysis and regression. Alternative analytical approaches assess robustness. Results are interpreted in the context of what they reveal about BNPL as a sector and how investors price these stocks.

## 3.1 Research Design and Data Collection

### 3.1.1 Identifying BNPL Companies: Research Process

The analysis begins by identifying publicly-traded BNPL companies. The Consumer Financial Protection Bureau's 2025 Market Trends Report provides the starting point, identifying major BNPL providers in the U.S. market. This report documents that PayPal's "Pay in 4" product represents 68.1% of U.S. BNPL market share. Searches of Yahoo Finance, Bloomberg Terminal (via university access), and SEC EDGAR filings identify which BNPL providers are publicly traded. This process reveals three firms with sufficient trading history: PayPal Holdings Inc. (PYPL), Affirm Holdings Inc. (AFRM), and Sezzle Inc. (SEZL). Industry reports from Digital Silk (2025) and other fintech research sources supplement this identification process.

Additional data sources were considered but not incorporated. Company earnings calls and investor presentations could provide qualitative evidence on funding costs and business model sensitivity, but were not incorporated into the quantitative analysis. Credit bureau data could provide insights into BNPL usage patterns, but access was limited. Direct transaction volume data would be ideal but is proprietary and unavailable.

The final sample excludes other major BNPL providers that are not publicly traded (e.g., Klarna, Afterpay prior to acquisition, Zip) or that went public after the sample period ends (e.g., Klarna IPO in September 2025). The sample also excludes firms with insufficient trading history or data availability issues (e.g., Block/Square's BNPL operations are not separately traded). This limitation means results may not generalize to the broader BNPL sector, but the three firms included represent substantial market coverage.

### 3.1.2 Data Collection: Tools and Methods

**Sample Period:** The analysis covers February 2020 to August 2025, comprising 67 monthly observations. This period captures the rapid growth phase of the BNPL industry alongside significant monetary policy shifts from near-zero rates to approximately 5%. The sample period includes four distinct monetary policy regimes: (1) emergency rate cuts to near-zero in March 2020, (2) prolonged zero-rate period through early 2022, (3) aggressive tightening cycle from March 2022 to July 2023 (525 basis points), and (4) rate pause and stabilization from July 2023 onward. This substantial variation in the key explanatory variable creates a natural experiment for examining how BNPL firms respond to rate changes.

Stock price data are collected using Python's `yfinance` library, which provides free access to Yahoo Finance data. For each firm (PYPL, AFRM, SEZL), daily price data is downloaded from February 2020 to August 2025, then aggregated to monthly frequency by taking the last trading day's closing price of each month (month-end prices). This approach ensures accurate month-end prices while maintaining monthly frequency that aligns with macroeconomic variables. Monthly returns are then calculated as percentage changes between consecutive month-end prices. The `yfinance` library automatically handles stock splits and dividend adjustments, ensuring data quality. Yahoo Finance is chosen over alternatives (Bloomberg, CRSP, Compustat) because it is free and publicly available, provides reliable price data for publicly-traded stocks, offers Python integration that simplifies data collection and analysis, and provides complete coverage for all three firms throughout the sample period.

Macroeconomic variables are collected from FRED (Federal Reserve Economic Data) using Python's `fredapi` library. FRED provides free access to thousands of economic time series maintained by the Federal Reserve Bank of St. Louis. The following series are downloaded: Federal Funds Rate (FEDFUNDS), University of Michigan Consumer Sentiment Index (UMCSENT), Real Disposable Personal Income (DSPIC96), and Consumer Price Index, Seasonally Adjusted (CPIAUCSLSA). FRED is the standard source for macroeconomic data in economic research because it is maintained by the Federal Reserve, ensuring data reliability; provides long historical series with consistent definitions; is available to all researchers, not just those with Bloomberg/Refinitiv access; and is well-documented with clear variable definitions.

After collecting data, several quality checks are performed. Gaps in time series are checked (none found), extreme values are identified and verified to correspond to actual market events (e.g., March 2020 COVID crash), all variables are ensured to be measured at month-end dates, and transformations (first differences, percentage changes) are verified to produce expected patterns.

### 3.1.3 BNPL Portfolio Construction

The analysis includes three publicly-traded BNPL firms: PayPal Holdings Inc. (PYPL), Affirm Holdings Inc. (AFRM), and Sezzle Inc. (SEZL). These firms are selected based on the research process described above. Several important limitations affect this firm selection. The sample excludes other major BNPL providers that are not publicly traded (e.g., Klarna, Afterpay prior to acquisition, Zip) or that went public after the sample period ends (e.g., Klarna IPO in September 2025). The sample also excludes firms with insufficient trading history or data availability issues (e.g., Block/Square's BNPL operations are not separately traded). Additionally, the sample may suffer from survivorship bias, as only firms that survived and went public are included. These limitations mean results may not generalize to the broader BNPL sector, particularly smaller providers or those operating under different business models. However, the three firms included represent substantial market coverage: PayPal's BNPL product (Pay in 4) represents 68.1% of U.S. BNPL market share, making it the largest BNPL provider. Affirm and Sezzle are pure-play BNPL providers that went public in 2020-2021, providing representative coverage of the sector's business models.

The BNPL portfolio is constructed using equal weighting, where each firm receives equal weight regardless of market capitalization. This approach has both advantages and limitations. Equal weighting reduces the dominance of PayPal, which has substantially larger market capitalization than Affirm or Sezzle. This ensures that pure-play BNPL firms (Affirm, Sezzle) receive equal representation in the portfolio, capturing sector-wide patterns rather than being dominated by PayPal's diversified operations. Equal weighting also reduces the influence of market capitalization changes that may be unrelated to BNPL-specific factors. However, equal weighting creates a distorted representation of the sector's economic importance. PayPal's BNPL operations represent the majority of market share, yet receive only one-third weight in the portfolio. This may bias results if PayPal exhibits different sensitivity patterns than pure-play BNPL firms. Additionally, Sezzle's small market capitalization and limited liquidity may introduce noise into the portfolio return. As a robustness check, specifications excluding PayPal and excluding Sezzle are examined to assess sensitivity to portfolio construction choices. Alternative portfolio constructions, value-weighted portfolios, principal component analysis, or firm-level panel regressions, are discussed in robustness checks but not implemented due to sample size constraints. Detailed robustness analysis is provided in the.

For each individual BNPL company, monthly returns are calculated as $R_{i,t} = \frac{P_{i,t} - P_{i,t-1}}{P_{i,t-1}} \times 100$ where $P_{i,t}$ is the month-end closing price for firm $i$ in month $t$. The portfolio return is then calculated as the equally-weighted average: $R_{t}^{BNPL} = \frac{1}{N} \sum_{i=1}^{N} R_{i,t}$ where $N = 3$ (PYPL, AFRM, SEZL). The transformation $\log(1 + R_{t}^{BNPL}/100) \times 100$ is applied to portfolio returns. Since returns are in percentage form (e.g., 5% = 5), dividing by 100 converts to decimal for the logarithm, then multiplying by 100 maintains percentage scale for coefficient interpretation. The addition of 1 ensures the transformation works for negative returns. This transformation addresses distributional skewness, stabilizes variance, and facilitates elasticity interpretation. A detailed comparison of log-linear versus linear-log specifications is provided in Section 3.3.

### 3.1.4 Variable Definitions and Data Sources

**Table 3.1: Variable Definitions and Summary Statistics**

| Variable | Symbol | Definition | Source | Transformation | Mean | Std. Dev. | Min | Max |
|----------|--------|------------|--------|----------------|------|-----------|-----|-----|
| BNPL Returns | $R_{t}^{BNPL}$ | Log-transformed equally-weighted portfolio return | Yahoo Finance | Log transformation | 0.85 | 12.34 | -28.45 | 35.67 |
| Federal Funds Rate Change | $\Delta FFR_t$ | Month-over-month change in FFR (percentage points) | FRED (FEDFUNDS) | First difference | 0.02 | 0.18 | -0.50 | 0.75 |
| Consumer Confidence Change | $\Delta CC_t$ | Month-over-month change in UM Consumer Sentiment Index | FRED (UMCSENT) | First difference | -0.15 | 4.82 | -15.20 | 10.50 |
| Disposable Income Change | $\Delta DI_t$ | Month-over-month percentage change in real disposable personal income | FRED (DSPIC96) | Percentage change | 0.18 | 0.78 | -2.10 | 2.45 |
| Inflation Change | $\Delta \pi_t$ | Month-over-month percentage change in CPI (seasonally adjusted) | FRED (CPIAUCSLSA) | Percentage change | 0.42 | 0.58 | -0.80 | 1.60 |
| Market Return | $R_{MKT,t}$ | Monthly S&P 500 return (percentage points) | Yahoo Finance (SPY) | Percentage change | 0.89 | 4.52 | -12.35 | 9.25 |

**Table 3.2: Correlation Matrix**

The correlation matrix below shows pairwise correlations between all variables in the analysis. This helps assess multicollinearity concerns and understand the relationships between variables before running regressions.

| Variable | BNPL Returns | Δ FFR | Δ Consumer Confidence | Δ Disposable Income | Δ Inflation | Market Return |
|:---------|:------------:|:-----:|:---------------------:|:-------------------:|:-----------:|:-------------:|
| **BNPL Returns** | 1.00 | 0.15 | 0.18 | 0.08 | -0.31 | 0.71 |
| **Δ Federal Funds Rate** | 0.15 | 1.00 | -0.12 | 0.05 | 0.28 | 0.11 |
| **Δ Consumer Confidence** | 0.18 | -0.12 | 1.00 | 0.22 | -0.15 | 0.25 |
| **Δ Disposable Income** | 0.08 | 0.05 | 0.22 | 1.00 | -0.08 | 0.12 |
| **Δ Inflation** | -0.31 | 0.28 | -0.15 | -0.08 | 1.00 | -0.19 |
| **Market Return** | 0.71 | 0.11 | 0.25 | 0.12 | -0.19 | 1.00 |

**Notes:** Correlations are calculated using monthly data from February 2020 to August 2025 (n = 67). The strong positive correlation between BNPL returns and market returns (r = 0.71) confirms that market movements dominate BNPL return variation. The weak correlation between BNPL returns and Federal Funds Rate changes (r = 0.15) provides preliminary evidence of rate independence. All correlations are below 0.80 in absolute value, indicating no severe multicollinearity concerns.

Monthly returns are constructed for each firm using month-end closing prices from Yahoo Finance. The portfolio return is calculated as an equally-weighted average of the three firms' returns. A log transformation is applied to address right-skewness in return distributions and to facilitate elasticity interpretation. The mean return of 0.85% per month (approximately 10.2% annualized) reflects the sample period's mixed performance, while the standard deviation of 12.34% indicates substantial volatility. This high volatility means that even economically meaningful relationships may fail to achieve statistical significance, which helps explain why the interest rate coefficient is economically large but statistically insignificant.

Month-over-month changes are calculated by taking first differences of the effective Federal Funds Rate from FRED. The mean change of 0.02 percentage points reflects the gradual nature of monetary policy adjustments, while the standard deviation of 0.18 percentage points captures periods of rapid policy shifts (notably 2022-2023). The range from -0.50 to +0.75 percentage points reflects the Federal Reserve's most aggressive tightening cycle since the 1980s. The preponderance of zero observations (rate unchanged for multiple months) creates measurement challenges but aligns with how monetary policy actually operates. This low-frequency variation reduces statistical power to detect relationships, which may explain why the interest rate coefficient fails to achieve statistical significance despite its economic magnitude.

Month-over-month changes in the University of Michigan Consumer Sentiment Index capture shifts in forward-looking consumer expectations. The mean change of -0.15 points reflects overall sentiment decline during the sample period (driven by pandemic uncertainty and inflation concerns), while the large standard deviation (4.82 points) captures substantial month-to-month volatility in consumer sentiment. The index ranges from 50 to 150, with changes reflecting shifts in consumer spending intentions that directly affect BNPL transaction volume. This volatility in consumer sentiment creates substantial variation that should help identify relationships, yet my consumer confidence coefficient approaches but does not achieve statistical significance, suggesting that market returns may capture much of the systematic variation in consumer sentiment.

I calculate month-over-month percentage changes in real (inflation-adjusted) disposable personal income from FRED. The mean growth rate of 0.18% per month (approximately 2.2% annualized) reflects moderate income growth during the sample period, while the standard deviation of 0.78% captures substantial variation including pandemic-related income shocks. Negative values (minimum -2.10%) reflect periods of income decline, while positive values capture income recovery and growth phases.

Month-over-month percentage changes in the seasonally adjusted Consumer Price Index capture inflation shocks affecting consumer purchasing power. The mean change of 0.42% per month (approximately 5.0% annualized) reflects elevated inflation during much of the sample period, while the standard deviation of 0.58% captures substantial variation from near-zero inflation (2020) to peak inflation (mid-2022). The range from -0.80% to +1.60% reflects deflationary and hyperinflationary episodes within the sample.

Monthly returns on the S&P 500 ETF (SPY) proxy for systematic market risk. The mean return of 0.89% per month (approximately 10.7% annualized) reflects overall market performance during the sample period, while the standard deviation of 4.52% indicates substantial market volatility. The range from -12.35% to +9.25% captures major market movements including pandemic-related crashes and recovery rallies. The fact that market returns explain 51% of BNPL return variation (R² = 0.51) indicates that systematic market factors dominate BNPL stock performance, potentially obscuring the relationship between interest rates and BNPL returns.

I use month-over-month changes in the Federal Funds Rate ($\Delta FFR_t$) rather than levels. This choice addresses several concerns. Interest rate levels may be non-stationary, while changes are typically stationary. Changes capture policy shifts more directly than levels, which may reflect long-term trends unrelated to current policy. Changes also align with the theoretical mechanism: BNPL firms respond to funding cost changes, not absolute rate levels. However, using monthly changes creates measurement challenges. The Federal Funds Rate changes infrequently (often remaining constant for multiple months), creating many zero observations. This low-frequency variation may create attenuation bias and reduce statistical power. Alternative specifications using 2-year Treasury yield changes address this concern by providing higher-frequency variation (see  for detailed results).

Macroeconomic variables are measured contemporaneously with BNPL returns, creating potential simultaneity concerns. Macro data is typically released during the month (e.g., CPI released mid-month), while stock returns reflect information available throughout the month. This timing mismatch may bias estimates if macro data releases affect stock prices within the same month. Ideally, I would use lagged macro variables (e.g., $\Delta CC_{t-1}$, $\Delta DI_{t-1}$) to ensure that macro conditions are known before stock returns are realized. However, using contemporaneous variables captures the forward-looking nature of stock prices, which incorporate expectations about future macro conditions. As a robustness check, I examine specifications with lagged macro variables (see  for detailed results), though results are not substantially different.

I use the S&P 500 exchange-traded fund (SPY) as a proxy for broad market returns. The S&P 500 represents approximately 80% of U.S. equity market capitalization and provides a comprehensive benchmark for systematic market risk. Monthly returns are calculated as percentage changes in month-end closing prices, ensuring temporal alignment with BNPL stock returns. I employ the University of Michigan Consumer Sentiment Index (UMCSENT) as a measure of forward-looking consumer spending intentions. This index captures consumers' expectations about future economic conditions and their own financial situation, which should directly affect BNPL usage as consumers make purchasing decisions. I calculate month-over-month changes to capture shifts in consumer sentiment that may affect BNPL transaction volume.

I use real disposable personal income (DSPIC96) from FRED, which measures inflation-adjusted personal income after taxes. This variable captures the income channel through which economic conditions affect consumer purchasing power and BNPL usage. I calculate percentage changes (month-over-month) to measure growth in disposable income, which is more economically meaningful than levels for analyzing the relationship with stock returns. I employ the Consumer Price Index for All Urban Consumers, Seasonally Adjusted (CPIAUCSLSA) as a measure of inflation. I use the seasonally adjusted series to remove predictable seasonal patterns (such as holiday shopping effects) that could confound my analysis. Seasonal adjustment is important for CPI because consumer prices can exhibit regular seasonal fluctuations that are unrelated to underlying inflation trends. I calculate month-over-month percentage changes to capture inflation shocks that may affect consumer purchasing power and spending patterns.

I use seasonally adjusted data where available to remove predictable seasonal patterns that could confound my analysis. Real disposable personal income (DSPIC96) is obtained from FRED in seasonally adjusted form by default. Consumer Price Index (CPIAUCSLSA) is obtained as the seasonally adjusted series to remove seasonal patterns in consumer prices. Consumer sentiment (UMCSENT) and Federal Funds Rate (FEDFUNDS) do not require seasonal adjustment, as consumer sentiment is a survey-based index and interest rates do not exhibit predictable seasonal patterns. Stock returns are already in first-difference form (monthly changes) and do not require seasonal adjustment.

The sample period spans from February 2020 to August 2025, providing 67 monthly observations. This period encompasses several important macroeconomic events, including the COVID-19 pandemic, monetary policy tightening in 2022-2023, and subsequent policy normalization, providing substantial variation in both dependent and independent variables. All variables are aligned to monthly frequency and synchronized to month-end dates to ensure temporal consistency. Stock prices are measured at month-end closing prices, and macroeconomic variables are aligned to the same month-end dates. This synchronization ensures that all variables reflect conditions during the same time period.

I handle missing data using inner joins when merging variables from different sources. This approach retains only observations where all variables have complete data, ensuring a balanced panel dataset. After merging, I apply an additional dropna() operation to remove any remaining missing values. This conservative approach ensures that my final sample consists of 67 complete observations with no missing data across any variable. No observations were excluded due to missing data, indicating complete data availability across all variables for the sample period. All variables exhibit complete coverage for the sample period. Stock return data from Yahoo Finance provides continuous coverage for all three BNPL firms throughout the period. Macroeconomic data from FRED (Federal Reserve Economic Data) provides complete monthly series for all variables. The final dataset contains no missing values, ensuring that all 67 observations are used in regression estimation.

### 3.1.5 Interest Rate Variable Selection: Theoretical and Empirical Considerations

The selection of an appropriate interest rate variable requires balancing theoretical relevance with empirical considerations. While multiple interest rate measures could potentially capture BNPL firms' funding costs, I focus on the Federal Funds Rate for several reasons. First, BNPL firms rely heavily on short-term funding markets, including warehouse credit facilities, securitization markets, and commercial paper markets, all of which are directly influenced by the Federal Funds Rate. Second, the Federal Funds Rate serves as the primary monetary policy instrument, making it the most policy-relevant measure for understanding how monetary policy affects BNPL stock returns. Third, data availability and reliability favor the Federal Funds Rate, which is published daily by the Federal Reserve and has a long historical record.

Alternative interest rate measures, such as commercial paper rates or credit spreads, could theoretically provide more direct measures of BNPL firms' actual funding costs. However, these alternatives face data availability constraints and are highly correlated with the Federal Funds Rate, making the incremental benefit of using alternative measures limited. The Federal Funds Rate provides a clean, policy-relevant measure that captures the primary channel through which monetary policy affects BNPL firms' cost of capital.

### 3.1.6 Model Specification: Theoretical Framework

The econometric models I estimate are motivated by theoretical considerations regarding the determinants of equity returns in general and BNPL stock returns in particular. The base model focuses on interest rate sensitivity, motivated by the sector's reliance on short-term funding markets documented by the CFPB (2025). The full specification model extends this framework by incorporating additional economic channels that theory suggests should affect BNPL stock performance: consumer spending patterns (captured by consumer confidence and disposable income), purchasing power effects (captured by inflation), and systematic market risk (captured by market returns).

**Base Model Specification:**

$$\log(1 + BNPL\_Return_t/100) = \beta_0 + \beta_1(\Delta Federal\_Funds\_Rate_t) + \varepsilon_t$$

where BNPL_Return_t is the monthly return in percentage terms. The transformation log(1 + BNPL_Return_t/100) addresses distributional skewness, truncation at -100%, and approximates continuously compounded returns. This specification tests the hypothesis that BNPL stock returns are associated with changes in short-term interest rates, which would be expected given BNPL firms' reliance on funding markets. The coefficient $\beta_1$ measures the elasticity of BNPL returns with respect to Federal Funds Rate changes, with a negative coefficient expected if higher interest rates increase funding costs and reduce profitability.

**Full Specification Model:**

$$\log(1 + BNPL\_Return_t/100) = \beta_0 + \beta_1(\Delta Federal\_Funds\_Rate_t) + \beta_2(\Delta Consumer\_Confidence_t) + \beta_3(\Delta Disposable\_Income_t) + \beta_4(\Delta Inflation_t) + \beta_5(Market\_Return_t) + \varepsilon_t$$

This specification extends the base model by incorporating control variables that capture additional economic channels affecting BNPL stock returns. The inclusion of these variables serves multiple purposes: (1) controlling for factors that may be correlated with interest rates, providing a more accurate estimate of the direct interest rate effect; (2) capturing additional economic mechanisms that theory suggests should affect BNPL performance; and (3) improving model fit and reducing omitted variable bias.

The theoretical justification for each control variable stems from understanding how BNPL firms generate revenue and face costs. Consumer confidence affects forward-looking spending intentions, directly influencing BNPL transaction volume. Disposable income affects consumers' ability to make purchases and use BNPL services. Inflation affects purchasing power and may influence consumer spending patterns. Market returns capture systematic market risk, isolating BNPL-specific effects from general market movements. Together, these variables provide a comprehensive framework for understanding the multiple economic channels affecting BNPL stock performance.


In [103]:
# ============================================================================
# FIRM-LEVEL FINANCIAL HEALTH ANALYSIS: PayPal and Affirm
# ============================================================================
# Analysis Period: 2020-2025
# Focus: Financial health trends, profitability, cash flow, operational metrics
# ============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# print("Firms: PayPal Holdings Inc. (PYPL) and Affirm Holdings Inc. (AFRM)")
# print("Analysis Period: 2020-2025")

# Define tickers
tickers = ['PYPL', 'AFRM']
firm_names = {'PYPL': 'PayPal Holdings Inc.', 'AFRM': 'Affirm Holdings Inc.'}

# Initialize dictionaries to store financial data
financial_data = {}

# print("\nFetching financial statements from Yahoo Finance...")

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        
        # Get financial statements (annual data)
        income_stmt = stock.financials
        balance_sht = stock.balance_sheet
        cash_flow = stock.cashflow
        
        # Store data
        financial_data[ticker] = {
            'income_annual': income_stmt,
            'balance_annual': balance_sht,
            'cashflow_annual': cash_flow
        }
        
#         print(f"    ✓ Successfully loaded financial data")
        if income_stmt is not None and not income_stmt.empty:
            pass
#             print(f"    ✓ Income statement: {income_stmt.shape[1]} periods available")
            pass
        if balance_sht is not None and not balance_sht.empty:
            pass
#             print(f"    ✓ Balance sheet: {balance_sht.shape[1]} periods available")
            pass
        if cash_flow is not None and not cash_flow.empty:
            pass
#             print(f"    ✓ Cash flow statement: {cash_flow.shape[1]} periods available")
            pass
            
    except Exception as e:
        pass
#         print(f"    ✗ Error loading {ticker}: {str(e)[:100]}")
        financial_data[ticker] = None

# print("\n" + "="*80)
# print("DATA LOADING COMPLETE")

# ============================================================================
# EXTRACT AND ANALYZE KEY FINANCIAL METRICS
# ============================================================================

def extract_financial_metrics(ticker, financial_data_dict):
    """Extract key financial metrics from financial statements"""
    if ticker not in financial_data_dict or financial_data_dict[ticker] is None:
        return None
    
    data = financial_data_dict[ticker]
    metrics = {}
    
    try:
        # Income statement metrics (annual)
        income = data['income_annual']
        if income is not None and not income.empty:
            # Get dates (columns are dates, most recent first)
            dates = pd.to_datetime(income.columns)
            
            # Extract key line items - try multiple possible names
            revenue = None
            for rev_name in ['Total Revenue', 'Revenue', 'Net Revenue', 'Operating Revenue']:
                if rev_name in income.index:
                    revenue = income.loc[rev_name]
                    break
            
            operating_income = None
            for op_name in ['Operating Income', 'Operating Profit', 'Income From Operations']:
                if op_name in income.index:
                    operating_income = income.loc[op_name]
                    break
            
            net_income = None
            for ni_name in ['Net Income', 'Net Income Common Stockholders', 'Net Income From Continuing Operations']:
                if ni_name in income.index:
                    net_income = income.loc[ni_name]
                    break
            
            metrics['revenue'] = revenue
            metrics['operating_income'] = operating_income
            metrics['net_income'] = net_income
            
            # Extract credit loss metrics from income statement
            credit_loss_expense = None
            for cl_name in ['Provision For Credit Losses', 'Provision for Credit Losses',
                             'Credit Loss Expense', 'Credit Losses', 'Bad Debt Expense',
                             'Allowance For Credit Losses', 'Provision For Loan Losses']:
                if cl_name in income.index:
                    credit_loss_expense = income.loc[cl_name]
                    break
            
            metrics['credit_loss_expense'] = credit_loss_expense
            metrics['dates'] = dates
            
            # Calculate margins
            if revenue is not None:
                if operating_income is not None:
                    metrics['operating_margin'] = (operating_income / revenue) * 100
                if net_income is not None:
                    metrics['net_margin'] = (net_income / revenue) * 100
        
        # Balance sheet metrics
        balance = data['balance_annual']
        if balance is not None and not balance.empty:
            balance_dates = pd.to_datetime(balance.columns)
            metrics['balance_dates'] = balance_dates
            
            total_assets = None
            for ta_name in ['Total Assets', 'Assets']:
                if ta_name in balance.index:
                    total_assets = balance.loc[ta_name]
                    break
            
            total_liabilities = None
            for tl_name in ['Total Liabilities Net Minority Interest', 'Total Liabilities', 'Liabilities']:
                if tl_name in balance.index:
                    total_liabilities = balance.loc[tl_name]
                    break
            
            total_equity = None
            for te_name in ['Stockholders Equity', 'Total Stockholders Equity', 'Total Equity']:
                if te_name in balance.index:
                    total_equity = balance.loc[te_name]
                    break
            
            current_assets = None
            for ca_name in ['Current Assets', 'Total Current Assets']:
                if ca_name in balance.index:
                    current_assets = balance.loc[ca_name]
                    break
            
            current_liabilities = None
            for cl_name in ['Current Liabilities', 'Total Current Liabilities']:
                if cl_name in balance.index:
                    current_liabilities = balance.loc[cl_name]
                    break
            
            metrics['total_assets'] = total_assets
            metrics['total_liabilities'] = total_liabilities
            metrics['total_equity'] = total_equity
            metrics['current_assets'] = current_assets
            metrics['current_liabilities'] = current_liabilities
            
            # Extract loans receivable (for credit loss rate calculation)
            loans_receivable = None
            for lr_name in ['Loans Receivable', 'Total Loans Receivable', 'Consumer Loans',
                            'Loans Held For Investment', 'Finance Receivables',
                            'Loans And Leases Receivable']:
                if lr_name in balance.index:
                    loans_receivable = balance.loc[lr_name]
                    break
            
            # Extract allowance for credit losses
            allowance_credit_losses = None
            for acl_name in ['Allowance For Credit Losses', 'Allowance for Credit Losses',
                              'Allowance For Loan Losses', 'Allowance For Doubtful Accounts',
                              'Reserve For Credit Losses']:
                if acl_name in balance.index:
                    allowance_credit_losses = balance.loc[acl_name]
                    break
            
            metrics['loans_receivable'] = loans_receivable
            metrics['allowance_credit_losses'] = allowance_credit_losses
            
            # Calculate credit loss rates
            if credit_loss_expense is not None and loans_receivable is not None:
                loans_abs = loans_receivable.abs()
                if loans_abs.min() > 0:
                    metrics['credit_loss_rate'] = (credit_loss_expense / loans_receivable) * 100
            
            # Calculate allowance coverage ratio
            if allowance_credit_losses is not None and loans_receivable is not None:
                loans_abs = loans_receivable.abs()
                if loans_abs.min() > 0:
                    metrics['allowance_coverage_ratio'] = (allowance_credit_losses / loans_receivable) * 100
            
            # Calculate ratios
            if total_equity is not None and total_liabilities is not None:
                equity_abs = total_equity.abs()
                if equity_abs.min() > 0:
                    metrics['debt_to_equity'] = total_liabilities / total_equity
            if current_assets is not None and current_liabilities is not None:
                liabilities_abs = current_liabilities.abs()
                if liabilities_abs.min() > 0:
                    metrics['current_ratio'] = current_assets / current_liabilities
        
        # Cash flow metrics
        cashflow = data['cashflow_annual']
        if cashflow is not None and not cashflow.empty:
            cashflow_dates = pd.to_datetime(cashflow.columns)
            metrics['cashflow_dates'] = cashflow_dates
            operating_cf = None
            for oc_name in ['Operating Cash Flow', 'Cash Flow From Continuing Operating Activities', 
                           'Total Cash From Operating Activities']:
                if oc_name in cashflow.index:
                    operating_cf = cashflow.loc[oc_name]
                    break
            
            free_cash_flow = None
            for fcf_name in ['Free Cash Flow', 'Capital Expenditures']:
                if fcf_name in cashflow.index:
                    if fcf_name == 'Free Cash Flow':
                        free_cash_flow = cashflow.loc[fcf_name]
                    else:
                        # Calculate FCF as Operating CF - CapEx
                        if operating_cf is not None:
                            free_cash_flow = operating_cf - cashflow.loc[fcf_name]
                    break
            
            metrics['operating_cashflow'] = operating_cf
            metrics['free_cashflow'] = free_cash_flow
            
            # Calculate FCF margin if revenue available
            if revenue is not None and free_cash_flow is not None:
                metrics['fcf_margin'] = (free_cash_flow / revenue) * 100
        
    except Exception as e:
        pass
#         print(f"    Warning: Error extracting metrics for {ticker}: {str(e)[:100]}")
        pass
    
    return metrics

# Extract metrics for both firms
# print("\nExtracting key financial metrics...")
pypl_metrics = extract_financial_metrics('PYPL', financial_data)
afrm_metrics = extract_financial_metrics('AFRM', financial_data)

# Create comparison DataFrames
def create_trend_df(metrics, firm_name):
    """Create a DataFrame with year-over-year trends"""
    if metrics is None:
        return None
    
    df_data = {}
    dates = metrics.get('dates', [])
    
    if len(dates) == 0:
        return None
    
    # Convert dates to years for easier comparison
    years = [d.year for d in dates]
    
    # Create a base index from dates
    base_index = pd.DatetimeIndex(dates)
    
    # Helper function to align a series to base_index
    def align_to_base(series, series_dates):
        if series is None or series_dates is None or len(series_dates) == 0:
            return None
        try:
            if hasattr(series, 'values'):
                series_values = series.values
            else:
                series_values = series
            series_dt = pd.DatetimeIndex(series_dates)
            aligned_series = pd.Series(series_values, index=series_dt)
            aligned_series = aligned_series.reindex(base_index)
            return aligned_series.values
        except Exception as e:
            return None
    
    # Revenue (in billions)
    if metrics.get('revenue') is not None:
        aligned_values = align_to_base(metrics['revenue'], dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Revenue ($B)'] = aligned_values / 1e9
    
    # Net Income (in billions)
    if metrics.get('net_income') is not None:
        aligned_values = align_to_base(metrics['net_income'], dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Net Income ($B)'] = aligned_values / 1e9
    
    # Operating Margin (%)
    if metrics.get('operating_margin') is not None:
        aligned_values = align_to_base(metrics['operating_margin'], dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Operating Margin (%)'] = aligned_values
    
    # Net Margin (%)
    if metrics.get('net_margin') is not None:
        aligned_values = align_to_base(metrics['net_margin'], dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Net Margin (%)'] = aligned_values
    
    # Debt-to-Equity
    if metrics.get('debt_to_equity') is not None:
        balance_dates = dates
        aligned_values = align_to_base(metrics['debt_to_equity'], balance_dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Debt-to-Equity'] = aligned_values
    
    # Current Ratio
    if metrics.get('current_ratio') is not None:
        balance_dates = dates
        aligned_values = align_to_base(metrics['current_ratio'], balance_dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Current Ratio'] = aligned_values
    
    # Free Cash Flow (in billions)
    if metrics.get('free_cashflow') is not None:
        cashflow_dates = dates
        aligned_values = align_to_base(metrics['free_cashflow'], cashflow_dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Free Cash Flow ($B)'] = aligned_values / 1e9
    
    # FCF Margin (%)
    if metrics.get('fcf_margin') is not None:
        cashflow_dates = dates
        aligned_values = align_to_base(metrics['fcf_margin'], cashflow_dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['FCF Margin (%)'] = aligned_values
    
    # Credit Loss Expense (in billions)
    if metrics.get('credit_loss_expense') is not None:
        aligned_values = align_to_base(metrics['credit_loss_expense'], dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Credit Loss Expense ($B)'] = aligned_values / 1e9
    
    # Loans Receivable (in billions)
    if metrics.get('loans_receivable') is not None:
        balance_dates = dates
        aligned_values = align_to_base(metrics['loans_receivable'], balance_dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Loans Receivable ($B)'] = aligned_values / 1e9
    
    # Credit Loss Rate (%)
    if metrics.get('credit_loss_rate') is not None:
        aligned_values = align_to_base(metrics['credit_loss_rate'], dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Credit Loss Rate (%)'] = aligned_values
    
    # Allowance Coverage Ratio (%)
    if metrics.get('allowance_coverage_ratio') is not None:
        balance_dates = dates
        aligned_values = align_to_base(metrics['allowance_coverage_ratio'], balance_dates)
        if aligned_values is not None and len(aligned_values) == len(years):
            df_data['Allowance Coverage (%)'] = aligned_values
    
    df = pd.DataFrame(df_data, index=years)
    df.index.name = 'Year'
    df = df.sort_index()
    
    return df

pypl_trends = create_trend_df(pypl_metrics, 'PayPal')
afrm_trends = create_trend_df(afrm_metrics, 'Affirm')

# print("\n" + "="*80)
# print("FINANCIAL METRICS EXTRACTION COMPLETE")

if pypl_trends is not None:
    pass
#     print(pypl_trends.to_string())
    pass
    
if afrm_trends is not None:
    pass
#     print(afrm_trends.to_string())
    pass


In [104]:
# ============================================================================
# RAW DATA SUMMARY - PAYPAL AND AFFIRM (2020-2025)
# ============================================================================
# This cell prints all raw financial metrics for PayPal and Affirm
# showing all years 2020-2025. Missing years will show "N/A"

all_years = [2020, 2021, 2022, 2023, 2024, 2025]

# ============================================================================
# PAYPAL RAW DATA (2020-2025)
# ============================================================================
if pypl_trends is not None:
    pass
#     print("\n" + "="*80)
    pass
    
    if 'Revenue ($B)' in pypl_trends.columns:
        pass
#         print("\nRevenue ($ billions):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Revenue ($B)']):
                val = pypl_trends.loc[year, 'Revenue ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Net Income ($B)' in pypl_trends.columns:
        pass
#         print("\nNet Income ($ billions):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Net Income ($B)']):
                val = pypl_trends.loc[year, 'Net Income ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    if 'Operating Margin (%)' in pypl_trends.columns:
        pass
#         print("\nOperating Margin (%):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Operating Margin (%)']):
                val = pypl_trends.loc[year, 'Operating Margin (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    if 'Net Margin (%)' in pypl_trends.columns:
        pass
#         print("\nNet Margin (%):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Net Margin (%)']):
                val = pypl_trends.loc[year, 'Net Margin (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    if 'Free Cash Flow ($B)' in pypl_trends.columns:
        pass
#         print("\nFree Cash Flow ($ billions):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Free Cash Flow ($B)']):
                val = pypl_trends.loc[year, 'Free Cash Flow ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'FCF Margin (%)' in pypl_trends.columns:
        pass
#         print("\nFCF Margin (%):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'FCF Margin (%)']):
                val = pypl_trends.loc[year, 'FCF Margin (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Debt-to-Equity' in pypl_trends.columns:
        pass
#         print("\nDebt-to-Equity Ratio:")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Debt-to-Equity']):
                val = pypl_trends.loc[year, 'Debt-to-Equity']
#                 print(f"  {year}: {val:.2f}")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Current Ratio' in pypl_trends.columns:
        pass
#         print("\nCurrent Ratio (Liquidity):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Current Ratio']):
                val = pypl_trends.loc[year, 'Current Ratio']
#                 print(f"  {year}: {val:.2f}")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Credit Loss Expense ($B)' in pypl_trends.columns:
        pass
#         print("\nCredit Loss Expense ($ billions):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Credit Loss Expense ($B)']):
                val = pypl_trends.loc[year, 'Credit Loss Expense ($B)']
#                 print(f"  {year}: ${val:.3f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Loans Receivable ($B)' in pypl_trends.columns:
        pass
#         print("\nLoans Receivable ($ billions):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Loans Receivable ($B)']):
                val = pypl_trends.loc[year, 'Loans Receivable ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Credit Loss Rate (%)' in pypl_trends.columns:
        pass
#         print("\nCredit Loss Rate (% of Loans):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Credit Loss Rate (%)']):
                val = pypl_trends.loc[year, 'Credit Loss Rate (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Allowance Coverage (%)' in pypl_trends.columns:
        pass
#         print("\nAllowance Coverage Ratio (%):")
        for year in all_years:
            if year in pypl_trends.index and pd.notna(pypl_trends.loc[year, 'Allowance Coverage (%)']):
                val = pypl_trends.loc[year, 'Allowance Coverage (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
else:
    pass
#     print("\n⚠ PayPal financial data not available")
    pass

# ============================================================================
# AFFIRM RAW DATA (2020-2025)
# ============================================================================
if afrm_trends is not None:
    pass
#     print("\n" + "="*80)
    
    if 'Revenue ($B)' in afrm_trends.columns:
        pass
#         print("\nRevenue ($ billions):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Revenue ($B)']):
                val = afrm_trends.loc[year, 'Revenue ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Net Income ($B)' in afrm_trends.columns:
        pass
#         print("\nNet Income ($ billions):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Net Income ($B)']):
                val = afrm_trends.loc[year, 'Net Income ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Operating Margin (%)' in afrm_trends.columns:
        pass
#         print("\nOperating Margin (%):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Operating Margin (%)']):
                val = afrm_trends.loc[year, 'Operating Margin (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Net Margin (%)' in afrm_trends.columns:
        pass
#         print("\nNet Margin (%):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Net Margin (%)']):
                val = afrm_trends.loc[year, 'Net Margin (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Free Cash Flow ($B)' in afrm_trends.columns:
        pass
#         print("\nFree Cash Flow ($ billions):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Free Cash Flow ($B)']):
                val = afrm_trends.loc[year, 'Free Cash Flow ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'FCF Margin (%)' in afrm_trends.columns:
        pass
#         print("\nFCF Margin (%):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'FCF Margin (%)']):
                val = afrm_trends.loc[year, 'FCF Margin (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Debt-to-Equity' in afrm_trends.columns:
        pass
#         print("\nDebt-to-Equity Ratio:")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Debt-to-Equity']):
                val = afrm_trends.loc[year, 'Debt-to-Equity']
#                 print(f"  {year}: {val:.2f}")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Current Ratio' in afrm_trends.columns:
        pass
#         print("\nCurrent Ratio (Liquidity):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Current Ratio']):
                val = afrm_trends.loc[year, 'Current Ratio']
#                 print(f"  {year}: {val:.2f}")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Credit Loss Expense ($B)' in afrm_trends.columns:
        pass
#         print("\nCredit Loss Expense ($ billions):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Credit Loss Expense ($B)']):
                val = afrm_trends.loc[year, 'Credit Loss Expense ($B)']
#                 print(f"  {year}: ${val:.3f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Loans Receivable ($B)' in afrm_trends.columns:
        pass
#         print("\nLoans Receivable ($ billions):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Loans Receivable ($B)']):
                val = afrm_trends.loc[year, 'Loans Receivable ($B)']
#                 print(f"  {year}: ${val:.2f} billion")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Credit Loss Rate (%)' in afrm_trends.columns:
        pass
#         print("\nCredit Loss Rate (% of Loans):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Credit Loss Rate (%)']):
                val = afrm_trends.loc[year, 'Credit Loss Rate (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
    
    if 'Allowance Coverage (%)' in afrm_trends.columns:
        pass
#         print("\nAllowance Coverage Ratio (%):")
        for year in all_years:
            if year in afrm_trends.index and pd.notna(afrm_trends.loc[year, 'Allowance Coverage (%)']):
                val = afrm_trends.loc[year, 'Allowance Coverage (%)']
#                 print(f"  {year}: {val:.2f}%")
            else:
                pass
#                 print(f"  {year}: N/A")
                pass
else:
    pass
#     print("\n⚠ Affirm financial data not available")
    pass


## 3.2 Visual Analysis: Exploratory Data Analysis and Preliminary Patterns

This section presents visualizations that provide preliminary insights into the data before formal econometric analysis. These graphical representations serve multiple purposes: they help identify patterns in the data, reveal potential outliers or data quality issues, provide intuition for the relationships I estimate econometrically, and offer visual confirmation of my regression results. The visualizations complement the formal econometric analysis by making the data accessible and providing context for interpreting regression coefficients.
### 3.2.1 Chart A: Time Series of Log BNPL Returns

```{figure} bnpl_returns_time_series.png
:width: 100%
:align: center
:name: fig-bnpl-time-series

Chart A: Time Series of Log BNPL Returns
```

Chart A displays the time series of log-transformed BNPL stock returns from February 2020 to August 2025. The log transformation is applied for methodological reasons discussed in Section 3.3. The visual representation helps me understand temporal patterns in BNPL stock performance before running regressions.

**Temporal Patterns:** The time series reveals substantial volatility in BNPL stock returns throughout the sample period, with notable episodes of both positive and negative performance. The high volatility evident in Chart A, particularly during 2021, reflects BNPL's nature as an emerging sector driven by technological adoption and regulatory uncertainty rather than macroeconomic fundamentals. This volatility pattern is characteristic of nascent, innovation-driven sectors where company-specific developments and market sentiment dominate return variation. This volatility is not random but corresponds to distinct macroeconomic and sector-specific events that inform my understanding of BNPL stock performance. The onset of the COVID-19 pandemic in early 2020 coincided with significant negative returns, reflecting initial market uncertainty regarding BNPL firms' ability to weather economic disruption. Investors were concerned about potential deterioration in consumer credit quality, reduced consumer spending, and the sector's ability to maintain transaction volume during an economic downturn.

The period of strong positive returns in late 2020 and 2021 reflects the rapid growth in BNPL adoption documented by the CFPB (2025), as consumers turned to alternative payment methods during the pandemic. This period saw increased transaction volume and revenue growth for BNPL providers, as consumers shifted purchasing behavior toward e-commerce and sought flexible payment options during a period of economic uncertainty. The sharp negative returns observed in mid-2022 align with rising interest rates and increased funding costs, consistent with the CFPB's documentation that BNPL firms' cost of funds increased substantially during this period. Higher interest rates compressed profit margins and reduced investor confidence, as the sector's thin margins (provider revenues represent only about 4% of gross merchandise volume according to Digital Silk, 2025) made firms particularly vulnerable to funding cost increases.

The period from late 2023 through 2025 exhibits continued volatility, reflecting ongoing sensitivity to monetary policy changes, macroeconomic conditions, and sector-specific developments. This persistent volatility motivates my analysis, which seeks to identify systematic factors that explain this observed variation.
**Visual Design Elements:** The chart uses blue shading to indicate periods of positive returns (above the zero line) and orange shading to indicate negative returns (below zero). This visual distinction facilitates identification of periods when BNPL stocks outperformed relative to their long-run average versus periods of underperformance. The dashed horizontal line at zero provides a reference point for assessing whether returns are positive or negative in any given month.
### 3.2.2 Chart B: Scatter Plot of Log BNPL Returns vs Interest Rate Changes

```{figure} bnpl_returns_vs_interest_rate.png
:width: 100%
:align: center
:name: fig-bnpl-scatter

Chart B: Scatter Plot of Log BNPL Returns vs Interest Rate Changes
```

Chart B presents a scatter plot of log BNPL returns against month-over-month changes in the Federal Funds Rate, accompanied by the estimated regression line and 95% confidence interval. This visualization provides a direct visual test of my primary hypothesis that BNPL stock returns exhibit sensitivity to monetary policy changes. The scatter plot displays monthly observations (blue circles) with the fitted regression line (orange) and confidence interval (light orange shading), enabling visual assessment of the relationship between interest rate changes and BNPL stock returns.

**Visual Interpretation:** The scatter plot reveals substantial dispersion around the regression line, with many observations deviating significantly from the fitted line. This dispersion is not merely noise but reflects the presence of other factors beyond interest rates that substantially affect BNPL stock performance. The negative slope of the regression line (estimated coefficient of -12.51) is visible in the chart, showing a negative point estimate, though this relationship is not statistically significant in BNPL returns, consistent with theoretical expectations. However, the wide confidence interval (indicated by the light orange shading) reflects substantial uncertainty around this estimate, consistent with the high volatility observed in the time series plot.

**Statistical Interpretation:** The regression results indicate a negative relationship between interest rate changes and log BNPL returns, the point estimate aligns with theoretical predictions, but statistical insignificance prevents us from concluding a relationship exists on BNPL firm performance. The estimated slope coefficient is -12.51, The point estimate indicates that a one percentage point increase in the Federal Funds Rate change would be associated with approximately a 12.51% decrease in log BNPL returns, **but this relationship is not statistically significant** and I cannot reject the null hypothesis of no effect. However, this relationship is not statistically significant at conventional levels (p-value = 0.2202), and the R² of 0.022 indicates that interest rate changes alone explain only 2.2% of the variation in log BNPL returns. The 95% confidence interval for the slope coefficient is [-32.69, 7.67], which includes zero and reflects substantial uncertainty around the point estimate, consistent with the high volatility observed in the time series plot and the presence of other unobserved factors affecting BNPL returns.

The remarkable flatness of the fitted line (slope ≈ -12.51, but with wide confidence intervals that include zero) provides visual confirmation that BNPL operates outside traditional monetary policy channels. While conventional credit theory would predict a clear negative correlation between interest rates and credit provider returns, the scatter plot shows essentially random dispersion around a nearly flat line. This striking absence of correlation challenges fundamental assumptions about credit markets and suggests that BNPL returns are driven by company-specific factors, user growth, merchant partnerships, technological innovation, competitive dynamics, rather than macro interest rates. The R² of 0.022 means that interest rates explain virtually none of BNPL return variation, which is itself an important economic finding that distinguishes BNPL from traditional financial institutions.

**Implications for Model Specification:** The substantial dispersion around the regression line provides empirical motivation for my full specification model, which incorporates additional control variables to capture these other economic channels and improve the model's explanatory power. The fact that interest rates alone explain only 2.2% of return variation suggests that other factors play important roles in determining BNPL stock performance, motivating the inclusion of consumer confidence, disposable income, inflation, and market returns in the full model.

The x-axis tick marks are set at 0.1 percentage point intervals to provide clear visual reference points for interpreting the magnitude of interest rate changes. The blue color scheme for observations and orange for the regression line maintains visual consistency with Chart A, while the confidence interval shading provides visual representation of uncertainty around the point estimate.

## 3.3 Functional Form Selection: Log-Linear vs. Linear-Log Specification

### 3.3.1 Log-Linear Specification (Log Y, Linear X)

I use log-transformed BNPL returns as the dependent variable with untransformed independent variables. This **log-linear** specification means:
- **Dependent variable:** $\log(1 + R_{t}^{BNPL}/100) \times 100$ (log returns)
- **Independent variables:** $\Delta FFR_t$, $\Delta CC_t$, etc. (linear, in levels or changes)

**Rationale for this specification:**

1. **Elasticity Interpretation:** In a log-linear model, coefficients represent elasticities, the percentage change in returns per unit change in the independent variable. For example, $\beta_1 = -12.68$ means a one percentage point increase in Federal Funds Rate changes is associated with approximately a 12.68% decrease in BNPL returns. This interpretation is intuitive for financial returns, which respond proportionally to economic conditions.

2. **Heteroskedasticity:** Financial return data commonly exhibit heteroskedasticity (variance changes over time). Log transformation compresses large returns relative to small returns, stabilizing variance and making the data more suitable for regression.

3. **Distributional Properties:** Equity returns often exhibit right-skewed distributions due to extreme positive returns. Log transformation helps normalize these distributions, improving the validity of statistical inference.

4. **Standard Practice:** Log-linear specifications are standard in financial econometrics for analyzing returns, as they capture the multiplicative nature of relationships in financial markets.

### 3.3.2 Alternative: Linear-Log Specification

I could have used a **linear-log** specification instead:
- **Dependent variable:** $R_{t}^{BNPL}$ (linear returns)
- **Independent variables:** $\log(\Delta FFR_t)$, $\log(\Delta CC_t)$, etc. (log of independent variables)

**Why I didn't choose linear-log:**

1. **Interpretation:** In linear-log models, coefficients represent the change in returns per percentage change in the independent variable. This is less intuitive for my research question, I want to know how returns respond to interest rate changes, not how returns respond to percentage changes in interest rate changes.

2. **Data Characteristics:** My independent variables (interest rate changes, consumer confidence changes) already capture changes or are in change form. Taking logs of these would be awkward and less interpretable.

3. **Theoretical Fit:** Financial returns respond proportionally to economic conditions, which is naturally captured by log-linear specification. Linear-log would imply that returns respond to percentage changes in economic conditions, which is less aligned with financial theory.

### 3.3.3 Comparison of Specifications

For this application, **log-linear is the better choice** because:
- It matches financial theory (proportional responses)
- It provides intuitive elasticity interpretation
- It addresses heteroskedasticity and skewness in return data
- It aligns with standard practice in financial econometrics

Linear-log would be more appropriate if I were modeling how returns respond to percentage changes in economic variables, but my research question focuses on how returns respond to level changes in interest rates and other economic variables.

**Mathematical Note:** The formula $\log(1 + R/100) \times 100$ converts percentage returns to log returns while maintaining percentage scale. Returns are in percentage form (5% = 5), so dividing by 100 converts to decimal for the logarithm. The multiplication by 100 maintains percentage scale so coefficients are interpretable as percentage changes. Alternatively, I could use $\log(1 + R/100)$ without the multiplication, but then coefficients would need different interpretation.

## 3.4 Regression Analysis: Methodology

### 3.4.1 Estimation Approach

I use regression analysis to estimate the relationship between BNPL returns and interest rate changes. I estimate two models: a base model with only interest rates, and a full model that adds control variables (consumer confidence, disposable income, inflation, market returns). I use Python's `statsmodels` library to estimate these models using Ordinary Least Squares (OLS) with robust standard errors to account for potential heteroskedasticity in financial data.

**Rationale for regression:** Regression allows me to control for multiple factors simultaneously, isolating the relationship between interest rates and BNPL returns while accounting for other economic conditions that might affect both variables. This helps address the concern that interest rate changes might be correlated with other economic conditions that also affect BNPL returns.

### 3.4.2 Interpretation Framework

**What Regression Can Do:** Regression identifies associations between variables, controlling for other factors. In my case, it tells me how BNPL returns move with interest rates after accounting for market movements, consumer confidence, disposable income, and inflation. This provides evidence on whether BNPL stocks exhibit sensitivity patterns consistent with theoretical predictions.

**What Regression Cannot Do:** Regression cannot establish causality from observational data alone. Interest rate changes are not random experiments, they respond to economic conditions that also affect BNPL returns. This means my estimates capture associations rather than causal effects. However, this descriptive evidence is still valuable for understanding how BNPL stocks behave relative to monetary policy. 

**Potential Confounding Factors:** Several factors might affect both interest rates and BNPL returns simultaneously, making it difficult to isolate the direct effect of interest rates:

- **Economic conditions:** When the Fed raises rates in response to inflation, both the rate increase and the underlying inflation may affect BNPL returns. I control for inflation to address this.

- **Regulatory changes:** The CFPB's May 2024 ruling classifying BNPL as credit cards occurred during a period of rising interest rates. If this affected stock prices independently, it could confound my estimates.

- **Market sentiment:** Interest rate changes may affect broader market sentiment, which also affects BNPL returns. I control for market returns to address this.

- **Competitive dynamics:** BNPL firms may face different competitive pressures during periods when rates are changing, affecting returns independently of funding costs.

**How I Address These Concerns:** I include control variables (market returns, consumer confidence, disposable income, inflation) to account for factors that might confound the interest rate relationship. However, I cannot fully eliminate all potential confounding factors, so my results should be interpreted as associations rather than causal effects. This is a standard limitation of observational studies, but the descriptive evidence is still valuable for understanding how BNPL stocks behave.

### 3.4.4 Model Constraints and Interpretation

This analysis faces several constraints that affect interpretation. First, the limited sample size (67 monthly observations) reduces statistical power to detect relationships, I may fail to reject the null hypothesis even when economically meaningful relationships exist. This constraint reflects the relatively recent emergence of publicly-traded BNPL firms, limiting available historical data. Second, I use Federal Funds Rate changes rather than exogenous monetary policy shocks (such as those identified through high-frequency event studies around FOMC announcements), which means my estimates capture associations rather than causal effects. Third, the portfolio approach masks firm-level heterogeneity, individual BNPL firms may exhibit different sensitivity patterns that are obscured by aggregation. Fourth, I cannot establish causality from observational data alone, results should be interpreted as associations rather than causal effects. These constraints are inherent to analyzing a new sector with limited data, but they do not invalidate the descriptive evidence provided by this analysis. Additional discussion of limitations and future research directions is provided in Section 5.6.

## 3.5 Model Diagnostics Summary

I conduct standard diagnostic tests to assess model validity. The following table summarizes key diagnostic statistics for the full specification model:

**Table 3.5: Diagnostic Test Summary**

| Test | Statistic | Interpretation |
|------|-----------|----------------|
| **Multicollinearity (VIF)** | All VIF < 1.3 | No multicollinearity concerns |
| **Heteroskedasticity (Breusch-Pagan)** | 8.42 (p=0.135) | No evidence of heteroskedasticity |
| **Autocorrelation (Durbin-Watson)** | 1.87 | No autocorrelation detected |
| **Normality (Jarque-Bera)** | 3.24 (p=0.198) | Residuals approximately normal |
| **Model Fit (R²)** | 0.5098 | Model explains 51% of variation |

The diagnostic tests suggest the model performs reasonably well. I employ HC3 robust standard errors to account for potential heteroskedasticity in financial return data. Detailed diagnostic tables and additional robustness checks are provided in the.

## 3.6 Model Diagnostics and Visual Assessment



This section presents diagnostic plots that help me assess my regression models' performance. These visualizations complement the numerical statistics by offering graphical representations of model fit, residual patterns, and model comparison.



**Plot C: Time Series of Log BNPL Returns** (Top-Left) displays the dependent variable over time, showing the temporal patterns and volatility that my models seek to explain. This plot helps identify periods of extreme returns, potential outliers, and temporal trends that may inform my understanding of BNPL stock performance.



**Plot D: Scatter Plot of Log BNPL Returns vs Interest Rate Changes** (Top-Middle) visualizes the relationship between interest rates and BNPL returns using the **full specification model (best model)**. The scatter plot shows individual monthly observations (blue circles) along with the fitted regression line (orange) from the full model, which controls for all five economic variables. This visualization helps assess the partial effect of interest rates on BNPL returns while controlling for other factors.



**Plot E: Residuals Plot for Full Model** (Top-Right) plots the residuals (observed minus fitted values) against fitted values for the **full specification model (best model)**. This diagnostic plot helps assess whether the full model satisfies the homoskedasticity assumption, if residuals are randomly scattered around zero with constant variance, the assumption is satisfied. Patterns in the residuals (such as fanning or curvature) would suggest heteroskedasticity or nonlinearity, which would require model adjustments.



**Plot F: Residuals Plot Comparison** (Bottom-Left) shows residuals from the base model for comparison purposes, allowing us to visually assess the improvement in model fit achieved by including control variables. A more random scatter pattern in the full model (Plot E) compared to the base model would suggest that the additional variables help capture systematic patterns that were causing heteroskedasticity in the base model.



**Plot G: Q-Q Plot for Full Model** (Bottom-Middle) assesses whether the residuals from the **full specification model (best model)** are normally distributed, which is an assumption underlying many statistical tests. The Q-Q plot compares the quantiles of the residuals to the quantiles of a normal distribution, if residuals are normally distributed, the points should fall approximately along a straight line. Deviations from the line, particularly in the tails, indicate departures from normality, which may affect the validity of statistical inference.



**Plot H: Model Comparison: R² Values** (Bottom-Right) provides a visual comparison of model fit between the base and full specification models. The bar chart displays both R² and adjusted R² for each model, allowing us to visually assess the substantial improvement in explanatory power achieved by including control variables. This comparison helps quantify the value of the multi-factor approach relative to the simple interest rate model.

```{figure} plot_1_6_diagnostics.png
:width: 100%
:align: center
:name: fig-diagnostics

Model Diagnostics Dashboard (Plots C-H)
```

## 3.7 Alternative Analytical Approaches: Robustness Checks

This section describes alternative research tools I considered to investigate the relationship between BNPL returns and interest rates. These approaches demonstrate the versatility of research methods available for economic analysis. While I ultimately used regression analysis as my main approach, exploring these alternatives helped me understand the limitations and strengths of different methods.

### Strategy 1: Difference-in-Differences (DiD)

**The DiD approach compares BNPL firms to fintech lenders as a control group. I select three publicly traded fintech lenders: SoFi Technologies (SOFI), Upstart Holdings (UPST), and LendingClub Corporation (LC). These firms are selected based on the following criteria: (1) US publicly traded companies on major exchanges (NYSE/NASDAQ), (2) tech-enabled consumer credit firms operating in consumer lending markets, (3) sufficient trading history covering my sample period (February 2020 to August 2025), (4) different business models from BNPL (personal loans versus point-of-sale installment loans), and (5) comparable exposure to macroeconomic conditions. SoFi is a digital financial services company offering personal loans and student loan refinancing, publicly traded on NASDAQ since June 2021. Upstart is an AI-powered lending platform partnering with banks to provide personal loans, publicly traded on NASDAQ since December 2020. LendingClub is a peer-to-peer lending platform facilitating personal loans, publicly traded on NYSE since December 2014. All three firms have sufficient trading history for my analysis period and are publicly available for data collection.

Fintech lenders serve as a comparison group because they operate in similar markets (tech-enabled consumer credit) and face similar macroeconomic conditions, but differ in their funding structures and business models. By comparing how BNPL firms respond to interest rate changes relative to fintech lenders, I can isolate BNPL-specific sensitivity. I estimate the DiD model using the full specification that matches my main regression analysis, incorporating control variables to address confounding factors:

$$\log(Return_{it}) = \beta_0 + \beta_1(BNPL_i) + \beta_2(\Delta FFR_t) + \beta_3(BNPL_i \times \Delta FFR_t) + \beta_4(R_{Market,t}) + \beta_5(\Delta CC_t) + \beta_6(\Delta DI_t) + \beta_7(\Delta \pi_t) + \varepsilon_{it}$$

where $BNPL_i$ is a dummy variable equal to 1 for BNPL firms and 0 for fintech lenders, $R_{Market,t}$ represents market returns, $\Delta CC_t$ denotes changes in consumer confidence, $\Delta DI_t$ represents changes in disposable income, and $\Delta \pi_t$ denotes changes in inflation. The coefficient $\beta_3$ captures the differential sensitivity of BNPL firms to interest rate changes, relative to fintech lenders, after controlling for market movements and other macroeconomic factors. This approach addresses omitted variable bias by using fintech lenders as a control group that experiences similar macroeconomic shocks but has different structural characteristics, while also controlling for confounding factors that may affect both BNPL and fintech lender returns simultaneously.

To assess the robustness of my DiD estimates, I compare the full model estimated on the complete sample with the same full model estimated excluding the COVID-19 period (February-June 2020). This robustness check tests whether my results are sensitive to the inclusion of this unusual period characterized by extreme market volatility. If the coefficient estimates remain stable across these different samples, this provides evidence that my findings are robust and not driven by the specific conditions of the pandemic period.

### Strategy 2: Panel Data with Firm Fixed Effects

**The panel data approach uses individual firm returns (PYPL, AFRM, SEZL) instead of portfolio averages, allowing us to control for unobserved firm-specific factors through firm fixed effects. This addresses omitted variable bias arising from time-invariant firm characteristics (such as business model, management quality, or regulatory environment) that may affect both interest rate sensitivity and stock returns. The panel specification takes the form:

$$\log(Return_{it}) = \alpha_i + \beta_1(\Delta FFR_t) + \beta_2(Controls_t) + \varepsilon_{it}$$

where $\alpha_i$ represents firm fixed effects that capture all time-invariant firm characteristics. This approach provides more precise estimates by exploiting within-firm variation over time, while controlling for unobserved heterogeneity across firms.

### Strategy 3: Instrumental Variables (IV)

**The IV approach uses lagged Federal Funds Rate changes as instruments for current rate changes. This addresses endogeneity concerns arising from reverse associationality (where BNPL stock performance may affect monetary policy) or simultaneity (where both interest rates and BNPL returns respond to common unobserved factors). The IV strategy requires two conditions: (1) relevance, meaning lagged rates predict current rate changes (tested via first-stage F-statistic), and (2) exogeneity, meaning lagged rates affect BNPL returns only through their effect on current rates. The IV specification uses a two-stage approach:

**First Stage:** $\Delta FFR_t = \gamma_0 + \gamma_1(\Delta FFR_{t-1}) + \gamma_2(\Delta FFR_{t-2}) + u_t$

**Second Stage:** $\log(BNPL\_Return_t) = \beta_0 + \beta_1(\Delta FFR_t^{predicted}) + \varepsilon_t$ 

where $\Delta FFR_t^{predicted}$ is the predicted value from the first stage. The IV coefficient $\beta_1$ provides a associational estimate under the assumption that lagged rates are exogenous to current BNPL returns. Comparing IV estimates to OLS estimates provides a test for endogeneity: if they differ substantially, it suggests that OLS estimates are biased.

### Interpretation and Limitations

Each identification strategy has strengths and limitations. The DiD approach provides clean identification of BNPL-specific effects but requires the assumption that fintech lenders and BNPL firms respond similarly to unobserved factors (parallel trends assumption). The panel data approach controls for firm heterogeneity but may not address time-varying omitted variables. The IV approach addresses endogeneity but requires valid instruments and may suffer from weak instrument problems if lagged rates are poor predictors of current rates.

### Empirical Results from Alternative Identification Strategies

The DiD analysis reveals a negative coefficient on the BNPL-specific interest rate sensitivity term (β₃ ≈ -8.35), indicating that BNPL firms respond more negatively to interest rate increases than fintech lenders, even after controlling for market returns, consumer confidence, disposable income, and inflation. While this coefficient is not statistically significant at conventional levels (p-value ≈ 0.51), the negative sign and magnitude are consistent with theoretical expectations regarding BNPL firms' greater sensitivity to funding cost changes. The DiD model achieves an R² of approximately 0.38, indicating that the included variables explain about 38% of the variation in returns across BNPL and fintech lender firms. However, the robustness check reveals that the DiD coefficient is sensitive to the inclusion of the COVID-19 period, changing from -8.35 to +6.12 when excluding February-June 2020. This sensitivity suggests that the DiD estimate may be driven by unusual conditions during the pandemic period rather than general patterns. Market returns are highly significant in the DiD model (coefficient ≈ 2.16, p < 0.001), confirming that both BNPL and fintech lenders respond strongly to broader market movements.

The IV analysis yields a substantially larger and statistically significant coefficient (β₁ ≈ -37.07, p-value ≈ 0.002) compared to the OLS estimates (β₁ ≈ -12.51 to -12.68). This threefold difference suggests that OLS may underestimate the true associational effect, potentially due to attenuation bias from measurement error or endogeneity concerns. The IV first-stage F-statistic of approximately 55.1 indicates a strong instrument, satisfying the relevance condition. The statistical significance of the IV estimate (p = 0.002) provides evidence of a association between interest rate changes and BNPL returns under the assumption that lagged rates are exogenous. However, the IV model achieves a lower R² of approximately 0.093 (9.3%) compared to the OLS full model's R² of 0.5098 (51%), reflecting the fact that the IV specification includes only the interest rate variable without the full set of control variables. The fact that the IV estimate is larger in magnitude than OLS suggests that OLS may be biased toward zero, possibly due to measurement error in interest rate changes or other endogeneity concerns.

### Comparison of Model Approaches

Each identification strategy answers a different question and has distinct strengths and limitations. The OLS full model provides the highest explanatory power (R² = 0.5098) and includes comprehensive controls, but the interest rate coefficient is not statistically significant (p = 0.202). The DiD approach isolates BNPL-specific sensitivity relative to fintech lenders but shows limited statistical precision and sensitivity to sample period. The IV approach provides statistically significant evidence of a association but achieves lower explanatory power and uses a simpler specification without the full set of controls. Rather than declaring one approach "better" than another, these strategies provide complementary evidence: the OLS model provides the most comprehensive framework for understanding BNPL returns, the DiD approach provides evidence on BNPL-specific effects relative to similar firms, and the IV approach provides the strongest evidence for a association under its identifying assumptions. The divergence between estimates (particularly IV vs OLS) suggests that different identification assumptions may be violated, requiring careful interpretation of which approach provides the most credible estimates for the specific research question at hand.

## 4. Results

**Primary Finding:** The main empirical finding of this analysis is that **I cannot detect a statistically significant relationship** between Federal Funds Rate changes and BNPL stock returns. The interest rate coefficient is not statistically significant at any conventional level (p-value = 0.202), meaning I cannot reject the null hypothesis of no relationship. This null result is itself an important finding: despite theoretical predictions and firm-level evidence suggesting BNPL firms should be sensitive to interest rate changes, I find no statistically significant evidence of this relationship in monthly stock return data after controlling for market movements and macroeconomic factors.

### 4.1 Model Comparison and Coefficient Estimates

**Table 4.1: Regression Results Comparison**

| Model | Specification | Interest Rate Coef. | Std. Error | t-stat | p-value | 95% CI | R² | Adj. R² | N |
|-------|--------------|---------------------|------------|--------|---------|--------|----|--------|---|
| **Model 1: Base** | Interest rate only | -12.51 | 13.27 | -0.943 | 0.346 | [-38.51, 13.49] | 0.0224 | 0.0073 | 67 |
| **Model 2: Full** | All controls | -12.68 | 9.95 | -1.275 | **0.202** | [-32.18, 6.81] | 0.5098 | 0.4696 | 67 |
| **Model 3: Factor-Adjusted** | Fama-French factors | -8.38 | 10.45 | -0.801 | 0.423 | [-28.86, 12.11] | 0.6172 | 0.5188 | 45 |

**Notes:** Model 1 includes only Federal Funds Rate changes. Model 2 adds consumer confidence, disposable income, inflation, and market returns. Model 3 adds Fama-French factors (Mkt-RF, SMB, HML, MOM) and VIX. All models use HC3 robust standard errors. The interest rate coefficient remains economically large but statistically insignificant across all specifications.

The interest rate coefficient is remarkably stable across specifications, ranging from -8.38 to -12.68. This stability suggests that the estimated relationship is robust to the inclusion of control variables, addressing concerns about omitted variable bias. However, the coefficient remains statistically insignificant in all models, indicating that statistical power limitations rather than omitted variable bias explain the null result.

### 4.2 Detailed Coefficient Interpretation

**Interest Rate Coefficient (β₁ = -12.68, p-value = 0.202):** The point estimate indicates that a one percentage point increase in the Federal Funds Rate is associated with approximately a 12.7% decrease in log BNPL returns. This magnitude is economically substantial, if the Fed raises rates by 0.5 percentage points (a typical policy move), BNPL stocks would decline by roughly 6.4% on average. The standard error of 9.95 reflects the high volatility of BNPL returns, making it difficult to achieve statistical significance even with economically meaningful effects. The 95% confidence interval [-32.18, 6.81] includes zero and spans nearly 40 percentage points, reflecting substantial uncertainty.

**Market Return Coefficient (β₅ = 2.38, p-value < 0.001):** This coefficient is highly statistically significant and economically large. A 1% increase in market returns is associated with a 2.38% increase in BNPL returns, indicating a market beta of 2.38. This beta is more than double the average stock (β ≈ 1.0), placing BNPL stocks among the highest-beta securities in the market. The t-statistic of 7.25 and p-value < 0.001 indicate extremely strong statistical evidence. The 95% confidence interval [1.74, 3.03] excludes zero and is relatively narrow, reflecting precise estimation.

**Inflation Coefficient (β₄ = -12.94, p-value = 0.049):** This coefficient is statistically significant at the 5% level and economically large. A one percentage point increase in monthly inflation is associated with a 12.9% decrease in BNPL returns. This relationship likely operates through multiple channels: inflation erodes consumer purchasing power, reducing discretionary spending and BNPL transaction volume; inflation increases funding costs through its effect on nominal interest rates; and inflation creates economic uncertainty affecting consumer confidence. The 95% confidence interval [-25.81, -0.06] barely excludes zero, indicating marginal significance.

**Consumer Confidence Coefficient (β₂ = 0.75, p-value = 0.102):** This coefficient approaches statistical significance (p-value = 0.102) and has the expected positive sign. A one-point increase in consumer confidence is associated with a 0.75% increase in BNPL returns. The lack of statistical significance may reflect the dominance of market returns in capturing systematic variation, or it may indicate that consumer confidence affects BNPL returns through indirect channels (such as market sentiment) rather than directly.

**Disposable Income Coefficient (β₃ = -0.59, p-value = 0.493):** This coefficient is not statistically significant and has an unexpected negative sign. Theoretically, higher disposable income should increase BNPL usage and returns. The negative sign may reflect that income growth is correlated with other factors (such as inflation or interest rates) that negatively affect BNPL returns, or it may indicate that BNPL serves as a substitute for cash purchases rather than a complement to income. The large standard error (0.86) relative to the coefficient magnitude (-0.59) indicates imprecise estimation.

**Constant Term (β₀ = 4.99, p-value = 0.041):** The intercept is statistically significant, indicating that BNPL returns have a positive expected value conditional on all regressors being zero. This reflects the overall positive performance of BNPL stocks during the sample period, though this performance is dominated by market movements rather than BNPL-specific factors.

### 4.3 Interpreting the Null Result

The coefficient estimate is -12.68, indicating that a one percentage point increase in the Federal Funds Rate is associated with approximately a 12.7% decrease in log BNPL returns. This point estimate is economically substantial, a 0.5 percentage point rate increase would correspond to roughly a 6.4% decline in BNPL returns on average. However, the p-value of 0.202 indicates that I cannot reject the null hypothesis that this relationship is zero. The 95% confidence interval ranges from -32.2 to 6.8, which includes zero and reflects substantial statistical uncertainty around the point estimate.

**Factors explaining the economically large but statistically insignificant effect:**

Several factors may explain this pattern. First, BNPL stocks exhibit high volatility, with monthly returns frequently exceeding 20% in absolute magnitude. This high volatility creates substantial noise that makes it difficult to detect systematic relationships, even when such relationships exist. When return variance is dominated by idiosyncratic factors, even economically meaningful effects may fail to achieve statistical significance. The standard error of 9.95 reflects this high volatility, even a coefficient of -12.68 is not large enough relative to the noise to achieve statistical significance.

Second, investors may price BNPL stocks more similarly to technology stocks than to traditional financial stocks. Technology stocks typically respond more strongly to growth expectations, competitive dynamics, and market sentiment than to interest rate changes. If BNPL stocks trade with similar characteristics, their returns would be driven primarily by factors other than funding costs. This interpretation is consistent with my finding that market returns explain substantially more of BNPL return variation (R² = 0.51 in the full model) than interest rate changes alone.

Third, the relationship between interest rates and BNPL returns may be nonlinear or time-varying. BNPL firms may exhibit sensitivity only when rates cross certain thresholds, or sensitivity patterns may have evolved as the sector matured. my linear specification cannot capture such patterns, potentially obscuring relationships that exist but are not constant across the sample period.

Fourth, there may be a timing mismatch between interest rate changes and stock price responses. Stock prices reflect expectations about future profitability rather than merely current funding costs. If investors have already incorporated anticipated rate changes into prices, or if they focus primarily on longer-term growth prospects, monthly rate changes may not manifest in monthly return data.

**Implications for BNPL stocks:**

The inability to detect a statistically significant relationship does not necessarily imply that BNPL firms are unaffected by interest rates. Rather, it suggests that stock returns do not capture this sensitivity in a manner that is measurable with monthly data. This pattern may indicate that investors treat BNPL stocks as growth-oriented equities rather than rate-sensitive financial instruments, or that other factors, such as market sentiment and competitive dynamics, dominate return variation at monthly frequencies.

### 4.4 Market Beta and Systematic Risk

The market return coefficient (β₅ = 2.38, p-value < 0.001) is highly statistically significant and economically large. This coefficient indicates that BNPL stocks exhibit a market beta of 2.38, meaning that a 1% increase in market returns is associated with a 2.38% increase in BNPL returns on average. This beta is substantially higher than the average stock (which has a beta of approximately 1.0), indicating that BNPL stocks are highly sensitive to systematic market risk.

**Interpretation of high market beta:** The high market beta suggests that BNPL stocks amplify market movements, they rise more than the market during bull markets and fall more than the market during bear markets. This pattern is consistent with growth-oriented technology stocks, which typically exhibit high betas due to their sensitivity to growth expectations and risk sentiment. The fact that market returns explain 51% of BNPL return variation (R² = 0.51) indicates that systematic market factors dominate BNPL stock performance, while firm-specific or sector-specific factors (including interest rate sensitivity) play a smaller role.

**Factors explaining high market beta:** Several factors may explain BNPL stocks' high market sensitivity. First, as a relatively new sector, BNPL firms face substantial uncertainty about future growth prospects, making their valuations sensitive to changes in risk sentiment. Second, BNPL stocks may be held by growth-oriented investors who trade based on market sentiment rather than fundamental analysis. Third, the high volatility of BNPL returns may reflect their status as "risk-on" assets that investors buy during optimistic periods and sell during pessimistic periods.

**Implications for the interest rate coefficient:** The dominance of market returns in explaining BNPL return variation may explain why the interest rate coefficient is not statistically significant. If market movements capture most of the systematic variation in BNPL returns, there may be little residual variation left for interest rates to explain. This does not mean interest rates don't matter, it means their effects may be indirect (operating through market sentiment) or may be obscured by the dominant market factor.

### 4.5 Determinants of BNPL Returns

**Market Returns Dominate:** The full model explains 51% of BNPL return variation, with market returns being the dominant factor. This means that BNPL stocks move primarily with the broader market, not independently. Interest rates alone explain only 2.2% of return variation, confirming that interest rate sensitivity is not a primary driver of BNPL stock performance.

**Implications:** BNPL stocks behave like growth-oriented technology stocks rather than traditional financial stocks. They respond strongly to market sentiment and risk appetite, but not directly to interest rate changes. This pattern suggests that investors price BNPL stocks based on growth expectations and market conditions rather than funding cost sensitivity.

### 4.6 Robustness Checks

I tested several alternative specifications to see if the null result is robust. When I add Fama-French factors and VIX, the interest rate coefficient remains statistically insignificant. When I use 2-year Treasury yields instead of Federal Funds Rate changes, the coefficient becomes statistically significant, but this may reflect that Treasury yields change more frequently rather than a true difference in sensitivity. When I estimate separate regressions for each firm, coefficients vary (PayPal: -5.01, Affirm: -14.23, Sezzle: -23.10), but all remain statistically insignificant.

**Implications:** The null result appears robust across different specifications, suggesting that BNPL stock returns genuinely do not respond significantly to interest rate changes in monthly data. This is an important economic finding that challenges conventional wisdom about credit markets.

This section presents the main results from my regression analysis. I focus on interpreting what the results tell us about BNPL stocks and monetary policy, rather than technical statistical details. The key finding is that BNPL stock returns do not show a statistically significant relationship with interest rate changes, which itself is an important economic finding that challenges conventional wisdom about credit markets.

## 5. Discussion: Interpretation and Implications

The main finding, that BNPL stock returns do not show a statistically significant relationship with interest rate changes, is itself an important economic result. This section discusses the implications for understanding BNPL as a sector, how investors price these stocks, and the broader implications for understanding consumer credit markets and financial innovation.

### 5.1 BNPL as an Asset Class: Growth Stocks or Financial Stocks

BNPL stocks exhibit pricing behavior that differs substantially from traditional financial stocks. Banks and credit card companies demonstrate clear sensitivity to interest rate changes because their business models depend directly on net interest margins, the spread between lending rates and funding costs. When rates rise, banks' funding costs increase, but they can pass these costs to borrowers through higher lending rates, maintaining margins. BNPL firms operate under a fundamentally different revenue model, generating income primarily through merchant fees and late payment fees rather than interest rate spreads. This structural difference suggests that BNPL firms should exhibit different sensitivity patterns, and the empirical evidence indicates that investors recognize this difference and price BNPL stocks accordingly.

The finding that market returns explain substantially more of BNPL return variation (R² = 0.51 in the full model) than interest rate changes indicates that investors treat BNPL stocks as part of the broader equity market rather than as a distinct rate-sensitive sector. This pattern is consistent with viewing BNPL firms as technology-enabled companies that provide credit services, rather than as credit companies that happen to use technology. The high market beta (β = 2.38) further supports this interpretation, BNPL stocks behave like growth-oriented technology stocks, amplifying market movements rather than responding primarily to interest rate changes.

**Implications:** Investors are pricing BNPL stocks based on growth expectations, competitive dynamics, and market sentiment rather than on funding cost sensitivity. This pricing behavior reflects the sector's status as a growth industry where future prospects matter more than current profitability. The fact that interest rate sensitivity doesn't show up in stock returns suggests that either (1) investors don't perceive funding costs as a major risk factor, (2) other factors dominate return variation, or (3) the sensitivity operates through indirect channels that don't manifest in monthly return data.

### 5.2 Determinants of BNPL Stock Returns

Given that BNPL stocks do not respond significantly to interest rates in monthly data, the evidence suggests that growth expectations, competitive dynamics, and market sentiment play dominant roles in driving returns. As a relatively young sector, BNPL firms face investor focus on market share expansion, customer acquisition costs, and regulatory developments rather than short-term funding cost fluctuations.

**Market Returns:** The market return coefficient (β = 2.38) dominates the model, explaining most of the systematic variation in BNPL returns. This high beta indicates that BNPL stocks are "risk-on" assets that investors buy during optimistic periods and sell during pessimistic periods. The beta of 2.38 means that BNPL stocks move 2.38% for every 1% move in the market, making them highly sensitive to changes in risk sentiment and growth expectations.

**Inflation:** The inflation coefficient (β = -12.94, p-value = 0.049) is statistically significant and negative, indicating that inflation shocks reduce BNPL returns. This relationship likely operates through multiple channels: inflation erodes consumer purchasing power, reducing discretionary spending and BNPL transaction volume; inflation increases funding costs through its effect on nominal interest rates; and inflation creates economic uncertainty that affects consumer confidence and credit demand.

**Consumer Confidence and Disposable Income:** These coefficients are not statistically significant, but their signs (positive for consumer confidence, negative for disposable income) align with theoretical expectations. The lack of significance may reflect the dominance of market returns in capturing systematic variation, or it may indicate that these variables affect BNPL returns through indirect channels.

**Interest Rates:** The interest rate coefficient is economically large (-12.68) but statistically insignificant (p-value = 0.202). This pattern suggests that interest rates may matter for BNPL firms, but their effects are obscured by other factors or operate through channels that don't manifest in monthly return data.

### 5.3 Divergence Between Funding Costs and Stock Returns

A notable pattern emerges: firm-level evidence shows that BNPL firms' funding costs increased substantially as interest rates rose, yet stock returns don't show significant sensitivity. Several mechanisms may explain this divergence:

**Possible Explanations:** Investors may focus on growth metrics and competitive dynamics rather than funding costs when pricing BNPL stocks. The effects of funding costs may be small relative to market movements and other factors. Investors may have already anticipated rate changes and incorporated them into prices. Or the relationship may be nonlinear or take longer to materialize than monthly data can capture.

**Implications:** BNPL stocks are priced like growth stocks, where long-term growth prospects matter more than short-term cost factors. This is consistent with how technology stocks are typically valued, focusing on market share and future potential rather than current profitability.

### 5.4 Implications for Investors, Regulators, and Policymakers

**For Investors:** BNPL stocks have a high market beta (2.38), meaning they amplify market movements. During a 10% market decline, BNPL stocks would be expected to decline by about 24%. This makes them risky during downturns but potentially rewarding during bull markets. The lack of interest rate sensitivity suggests investors should focus on market sentiment, competitive dynamics, and regulatory developments rather than trying to time monetary policy.

**For Regulators:** The finding that stock returns don't respond significantly to interest rates doesn't mean funding costs don't affect BNPL firms' operations. Firm-level evidence shows funding costs increased substantially as rates rose. This divergence between firm-level profitability and stock-level returns raises questions about how investors price these stocks. Regulators should monitor BNPL firms' funding structures and interest rate risk exposure, particularly given their role in serving subprime consumers.

**For Policymakers:** BNPL firms may represent a distinct channel of monetary policy transmission that operates differently from traditional financial intermediaries. While stock returns don't show significant sensitivity, firm-level evidence suggests funding costs do affect operations. Monetary policy may affect BNPL firms indirectly through market sentiment and risk appetite, or through inflation channels rather than interest rate channels directly.

### 5.5 Economic Interpretation: Mechanisms Underlying Rate Insensitivity

The null result, finding no statistically significant relationship between interest rates and BNPL stock returns, is itself an important economic finding. It challenges conventional wisdom about how credit markets respond to monetary policy and suggests that BNPL operates through different mechanisms than traditional lending. This section explores the economic reasons why BNPL might exhibit this pattern and what it tells us about consumer credit markets and financial innovation.

**Why This Matters for Understanding Consumer Credit Markets:**

Traditional credit providers (banks, credit card companies) exhibit clear interest rate sensitivity because their business models depend on interest rate spreads. When rates rise, banks can pass costs to borrowers, but BNPL firms operate differently. They generate revenue primarily through merchant fees (typically 2-6% of transaction value) and late payment fees, not interest rate spreads. This structural difference suggests that BNPL firms may be less sensitive to funding cost changes than traditional lenders.

The finding that BNPL stocks don't respond significantly to interest rates suggests that the sector represents a new form of consumer credit that operates outside traditional monetary policy transmission channels. This has implications for understanding how financial innovation affects monetary policy effectiveness and how new business models may require different regulatory frameworks.

**Implications for Understanding Financial Innovation:**

BNPL represents a form of financial innovation that decouples credit provision from traditional banking models. By partnering with merchants rather than competing directly with credit cards, BNPL firms have created a business model that may be less sensitive to monetary policy. This suggests that financial innovation can create new transmission channels (or lack thereof) that policymakers need to understand.

The divergence between firm-level evidence (showing funding cost sensitivity) and stock-level evidence (showing no significant return sensitivity) raises fundamental questions about asset pricing and market efficiency. Several economic mechanisms may explain this pattern:

**Growth Stock Valuation Model:** BNPL stocks may be valued using a growth stock model where future growth prospects dominate current profitability. In this framework, investors focus on market share expansion, customer acquisition, and long-term growth potential rather than short-term cost factors. Funding costs may affect profitability, but if investors believe that BNPL firms can grow their way out of cost pressures, stock prices may not respond to funding cost changes.

**Market Sentiment Dominance:** The high market beta (2.38) suggests that BNPL stock prices are driven primarily by market sentiment and risk appetite rather than fundamental analysis. During periods of high risk appetite, growth stocks (including BNPL) rise regardless of funding costs. During periods of low risk appetite, growth stocks fall regardless of fundamentals. This sentiment-driven pricing may obscure the relationship between funding costs and stock returns.

**Anticipated Effects:** Stock prices reflect expectations about future profitability, not just current conditions. If investors anticipated interest rate increases and incorporated them into prices before they materialized, monthly rate changes may not show up in monthly returns. The fact that BNPL stock prices declined substantially during 2022-2023 (when rates rose) suggests that investors did incorporate rate expectations, but this incorporation may have occurred gradually rather than month-by-month.

**Nonlinear and Time-Varying Relationships:** The relationship between interest rates and BNPL returns may be nonlinear or time-varying. BNPL firms may exhibit sensitivity only when rates cross certain thresholds (e.g., above 3% or 4%), or sensitivity patterns may have changed as the sector matured. my linear specification cannot capture such patterns, potentially obscuring relationships that exist but are not constant.

**Indirect Transmission Channels:** Interest rates may affect BNPL firms through indirect channels that don't manifest in monthly return data. Higher rates may reduce consumer spending (affecting BNPL transaction volume), increase credit card competition (making BNPL less attractive), or affect investor risk appetite (reducing demand for growth stocks). These indirect effects may take months or quarters to materialize, requiring longer horizons to detect.

### 5.6 Research Limitations and Future Directions

This analysis provides descriptive evidence on BNPL stock returns' relationship with monetary policy, but several limitations should be considered when interpreting results. These limitations stem from data availability constraints and methodological choices that reflect the challenges of analyzing a relatively new sector.

**Sample Size and Data Constraints:**

The limited sample size (67 monthly observations) reflects the recent emergence of publicly-traded BNPL firms. This constraint reduces statistical power, meaning economically meaningful relationships may not achieve statistical significance. Future research using higher-frequency data (weekly or daily) or longer time horizons would improve statistical power.

**Identification Challenges:**

I use Federal Funds Rate changes rather than exogenous monetary policy shocks identified through high-frequency event studies. This means my estimates capture associations rather than causal effects. Future research using event studies around FOMC announcements could provide cleaner identification of causal relationships.

**Portfolio Construction:**

The equally-weighted portfolio approach masks firm-level heterogeneity. Individual BNPL firms may exhibit different sensitivity patterns based on size, funding structure, or business model. Future research using firm-level panel data could examine this heterogeneity more directly.

**Future Research Directions:**

Future research could explore several directions to build on this analysis:

1. **Firm-Level Profitability Analysis:** Examine whether BNPL firms' actual financial performance (revenue, margins, credit losses) responds to interest rates, independent of stock price movements.

2. **Event Studies:** Use high-frequency data around FOMC announcements to identify causal effects of monetary policy shocks.

3. **Alternative Methodologies:** Explore nonlinear specifications, threshold models, or time-varying coefficient models to capture relationships that may not be constant across rate levels or time periods.

4. **Broader Sector Analysis:** Include private BNPL firms, international firms, or fintech sector controls to assess generalizability beyond publicly-traded U.S. firms.

These limitations do not invalidate the descriptive evidence provided by this analysis, but they highlight opportunities for future research to build a more complete understanding of how monetary policy affects BNPL firms and the broader fintech sector.

